In [1]:
import random
import numpy as np
from faker import Faker
import pandas as pd
from datetime import date, timedelta

# Inicializar o Faker para dados em Português
faker = Faker('pt_PT')

In [2]:
Departamentos = ["Recursos Humanos", "Tecnologia da Informação", "Financeiro", "Marketing", "Vendas", "Qualidade", "Atendimento ao Cliente", "Jurídico"]
Cargos = ["Gerente de Projetos", "Analista de Dados", "Desenvolvedor de Software Sénior", "Desenvolvedor de Software Júnior", "Engenheiro de DevOps","Designer UI/UX","Analista de Sistemas","Técnico de Suporte de TI","Analista de Recursos Humanos", "Especialista em Recrutamento", "Gerente de Marketing", "Especialista em Marketing Digital", "Analista de Mídia Social","Redator (Copywriter)","Gerente de Vendas","Executivo de Contas","Representante de Vendas Interno","Analista Financeiro","Contador","Gerente Financeiro","Analista de Qualidade","Inspetor de Qualidade","Agente de Atendimento ao Cliente","Supervisor de Atendimento ao Cliente",  "Especialista em Sucesso do Cliente",  "Advogado Corporativo","Assistente Jurídico","Assistente Administrativo","Recepcionista","Diretor de Operações (COO)"]

In [3]:
def criar_departamentos():
    lista_departamentos = []
    for i in range(1, len(Departamentos) + 1):
        lista_departamentos.append({
            "ID_depart": i,
            "Nome": Departamentos[i-1],
            "ID_gerente": None # Começa como Nulo
        })
    return pd.DataFrame(lista_departamentos)



In [4]:
def criar_funcionarios(n=50):
    lista_funcionarios = []
    for i in range(1, n + 1):
        lista_funcionarios.append({
            "ID_fun": i,
            "NIF": str(283684274 + i),
            "Primeiro_nome": faker.first_name(),
            "Ultimo_nome": faker.last_name(),
            "Nome_rua": faker.street_name(),
            "Nome_localidade": faker.city(),
            "Codigo_postal": faker.postcode(),
            "Num_telemovel": faker.phone_number(),
            "Email": faker.unique.email(),
            "Data_nascimento": faker.date_of_birth(minimum_age=18, maximum_age=65),
            "Cargo": random.choice([cargo for cargo in Cargos if cargo != "Gerente"]),
            "ID_depart": np.random.randint(1, len(Departamentos) + 1)
        })
    return pd.DataFrame(lista_funcionarios)

In [5]:
Departamentos = criar_departamentos()
Funcionarios = criar_funcionarios(100)
Departamentos["ID_gerente"] = random.sample(list(Funcionarios["ID_fun"]), len(Departamentos))



In [6]:
Departamentos


,ID_depart,Nome,ID_gerente
0,1,Recursos Humanos,1
1,2,Tecnologia da Informação,14
2,3,Financeiro,78
3,4,Marketing,25
4,5,Vendas,27
5,6,Qualidade,68
6,7,Atendimento ao Cliente,92
7,8,Jurídico,80


In [7]:
Funcionarios


,ID_fun,NIF,Primeiro_nome,Ultimo_nome,Nome_rua,Nome_localidade,Codigo_postal,Num_telemovel,Email,Data_nascimento,Cargo,ID_depart
0,1,283684275,Raquel,Mendes,Alameda João Glama,Caniço,5983-395,(351) 920 663 136,evagomes@example.net,2000-12-22,Executivo de Contas,4
1,2,283684276,Micael,Gomes,Avenida Reis,Penafiel,5966-936,(351) 920 522 032,epinho@example.com,1979-07-22,Representante de Vendas Interno,7
2,3,283684277,César,Cunha,Av de Batista,Rebordosa,2135-320,+351271824088,clara52@example.net,1991-12-06,Analista de Qualidade,7
3,4,283684278,Carlota,Domingues,Alameda Paris (Nascente),Santo Tirso,7091-830,(351) 928601229,florpires@example.org,1977-05-09,Analista Financeiro,5
4,5,283684279,Valentim,Torres,Avenida de Borges,Peniche,1788-977,+351236833846,icosta@example.org,1995-08-02,Desenvolvedor de Software Júnior,5
...,...,...,...,...,...,...,...,...,...,...,...,...
95,96,283684370,Jorge,Paiva,Praça Correia,Olhão,7393-035,939618137,soareslourenco@example.org,1982-04-01,Analista de Dados,2
96,97,283684371,Artur,Valente,Largo Wilson Paiva,Rio Maior,6701-347,922648158,zsantos@example.com,1967-11-14,Desenvolvedor de Software Sénior,2
97,98,283684372,Kévim,Campos,Alameda dos Navegadores,Câmara de Lobos,1901-816,(351) 247082297,marcosfernandes@example.com,1960-02-15,Especialista em Sucesso do Cliente,1
98,99,283684373,Tomé,Batista,Travessa de Paiva Couceiro,Gandra,7227-779,(351) 919 357 554,eramos@example.com,1978-07-15,Assistente Administrativo,6


In [8]:
def criar_remuneracoes_salarios_beneficios(funcionarios_df):
    """
    Gera um HISTÓRICO de dados para Remuneracoes, Salario e Beneficios.
    Garante que nenhuma Data_fim ultrapassa 2025-12-31.
    """
    MAX_DATE = date(2025, 12, 31)
    lista_remuneracoes = []
    lista_salarios = []
    lista_beneficios = []

    tipos_de_beneficios = ['Subsídio Alimentação', 'Seguro Saúde', 'Carro Empresa', 'Subsídio Transporte', 'Telemóvel Empresa']

    for index, funcionario in funcionarios_df.iterrows():
        id_fun = funcionario['ID_fun']
        
        num_registos_historicos = random.randint(1, 4)
        
        salario_atual = round(np.random.normal(loc=1300, scale=400), 2)
        if salario_atual < 950: salario_atual = 950
            
        # Garantir que o primeiro contrato começa, no máximo, no final de 2024
        max_start_date = date(2024, 12, 31)
        data_inicio_atual = faker.date_between(start_date='-5y', end_date=max_start_date)

        for i in range(num_registos_historicos):
            
            # Se a data de início já passou de 2025, não criamos mais registos
            if data_inicio_atual > MAX_DATE:
                break
                
            data_inicio_registo = data_inicio_atual
            proxima_data_inicio = None
            
            if i < num_registos_historicos - 1:
                # Se NÃO for o último registo (histórico)
                proxima_data_inicio = data_inicio_registo + timedelta(days=random.randint(350, 400))
                data_fim_registo = proxima_data_inicio - timedelta(days=1)
                
                # --- APLICA A REGRA DE DATA MÁXIMA ---
                if data_fim_registo > MAX_DATE:
                    data_fim_registo = MAX_DATE
                    proxima_data_inicio = MAX_DATE + timedelta(days=1) # Força o próximo a parar
            
            else:
                # Se FOR o último registo (o atual)
                if random.random() < 0.5:
                    # Tem uma data de fim. Calcular e limitar a 2 anos (para não ir muito longe)
                    data_fim_calculada = data_inicio_registo + timedelta(days=random.randint(90, 730))
                    
                    # --- APLICA A REGRA DE DATA MÁXIMA ---
                    if data_fim_calculada > MAX_DATE:
                        data_fim_registo = MAX_DATE
                    else:
                        data_fim_registo = data_fim_calculada
                else:
                    data_fim_registo = None # 50% de chance de ser Nulo
            
            # --- 1. Adicionar registo de Remuneração ---
            remuneracao_record = {
                "ID_fun": id_fun,
                "Data_inicio": data_inicio_registo,
                "Data_fim": data_fim_registo
            }
            lista_remuneracoes.append(remuneracao_record)
            
            # --- 2. Adicionar registo de Salário (versão simplificada) ---
            salario_record = {
                "ID_fun": id_fun,
                "Data_inicio": data_inicio_registo,
                "salario_bruto": round(salario_atual, 2)
            }
            lista_salarios.append(salario_record)
            
            # --- 3. Adicionar Benefícios (versão simplificada) ---
            if random.random() < 0.8:
                num_beneficios = random.randint(1, 3)
                beneficios_para_este_fun = random.sample(tipos_de_beneficios, num_beneficios)
                
                for tipo_beneficio in beneficios_para_este_fun:
                    beneficio_record = {
                        "ID_fun": id_fun,
                        "Data_inicio": data_inicio_registo,
                        "tipo": tipo_beneficio,
                        "valor": round(random.uniform(30, 450), 2)
                    }
                    lista_beneficios.append(beneficio_record)
            
            # --- 4. Preparar a próxima iteração ---
            salario_atual *= random.uniform(1.02, 1.10) # Aumento
            if proxima_data_inicio:
                data_inicio_atual = proxima_data_inicio

    remuneracoes_df = pd.DataFrame(lista_remuneracoes)
    salarios_df = pd.DataFrame(lista_salarios)
    beneficios_df = pd.DataFrame(lista_beneficios)
    
    return remuneracoes_df, salarios_df, beneficios_df

In [9]:
Remuneracoes, Salarios, Beneficios = criar_remuneracoes_salarios_beneficios(Funcionarios)

In [10]:
Remuneracoes

,ID_fun,Data_inicio,Data_fim
0,1,2021-09-02,2022-09-12
1,1,2022-09-13,2024-08-10
2,2,2023-03-27,2024-04-24
3,2,2024-04-25,2025-05-26
4,2,2025-05-27,2025-12-31
...,...,...,...
236,98,2022-08-05,None
237,99,2022-09-13,2023-08-28
238,99,2023-08-29,2025-06-07
239,100,2024-05-16,2025-06-14


In [11]:
Salarios

,ID_fun,Data_inicio,salario_bruto
0,1,2021-09-02,1474.38
1,1,2022-09-13,1544.13
2,2,2023-03-27,1453.93
3,2,2024-04-25,1596.32
4,2,2025-05-27,1732.50
...,...,...,...
236,98,2022-08-05,1492.75
237,99,2022-09-13,2346.42
238,99,2023-08-29,2426.28
239,100,2024-05-16,1207.09


In [12]:
Beneficios

,ID_fun,Data_inicio,tipo,valor
0,1,2022-09-13,Telemóvel Empresa,376.16
1,1,2022-09-13,Seguro Saúde,271.96
2,2,2023-03-27,Subsídio Transporte,76.42
3,2,2023-03-27,Subsídio Alimentação,443.36
4,2,2024-04-25,Carro Empresa,230.05
...,...,...,...,...
361,99,2023-08-29,Subsídio Transporte,51.20
362,100,2024-05-16,Carro Empresa,46.82
363,100,2025-06-15,Subsídio Alimentação,269.16
364,100,2025-06-15,Subsídio Transporte,432.21


In [13]:
def criar_ferias(funcionarios_df):
    """
    NÃO TEM O NUM_DIAS_PERMITIDOS
    Gera um histórico de férias para os funcionários.
    Cada funcionário pode ter entre 0 a 4 registos de férias.
    """
    lista_ferias = []
    estados_possiveis = ['Aprovado', 'Aprovado', 'Aprovado', 'Aprovado', 'Rejeitado', 'Por aprovar']
    
    for index, funcionario in funcionarios_df.iterrows():
        id_fun = funcionario['ID_fun']
        
        num_periodos_ferias = random.randint(0, 4)
        
        for _ in range(num_periodos_ferias):
            data_inicio = faker.date_between(start_date='-3y', end_date='today')
            num_dias = random.randint(2, 15)
            data_fim = data_inicio + timedelta(days=num_dias)
            
            ferias_record = {
                "ID_fun": id_fun,
                "data_inicio": data_inicio,
                "data_fim": data_fim,
                "num_dias": num_dias,
                "estado_aprov": random.choice(estados_possiveis)
            }
            lista_ferias.append(ferias_record)
            
    return pd.DataFrame(lista_ferias)

In [14]:
Ferias = criar_ferias(Funcionarios)
Ferias

,ID_fun,data_inicio,data_fim,num_dias,estado_aprov
0,1,2024-02-28,2024-03-06,7,Aprovado
1,1,2023-04-27,2023-05-10,13,Rejeitado
2,1,2023-07-24,2023-08-08,15,Aprovado
3,2,2025-03-31,2025-04-15,15,Aprovado
4,4,2023-04-27,2023-05-02,5,Por aprovar
...,...,...,...,...,...
204,96,2023-07-01,2023-07-09,8,Rejeitado
205,97,2023-05-20,2023-06-03,14,Aprovado
206,97,2023-05-08,2023-05-15,7,Rejeitado
207,97,2025-07-06,2025-07-13,7,Aprovado


In [15]:
def criar_dependentes(funcionarios_df):
    """
    Gera uma lista de dependentes e depois limpa os dados para garantir que
    nenhum funcionário tem mais do que um dependente com o parentesco 'Pai/Mãe'.

    Regras adicionadas:
    - 'Pai/Mãe' tem sempre data de nascimento ANTERIOR à do funcionário (>= 18 e <= 50 anos mais velho, limitado a 100 anos de idade total)
    - 'Filho(a)' tem sempre data de nascimento POSTERIOR à do funcionário (pelo menos 1 dia depois)
    """
    lista_dependentes = []
    # Usamos 'Pai/Mãe' como uma única categoria para simplificar
    parentescos_possiveis = ['Filho(a)', 'Cônjuge', 'Enteado(a)', 'Pai/Mãe', 'Filho(a)']  # Repetimos 'Filho(a)' para aumentar a probabilidade
    sexos_possiveis = ['Masculino', 'Feminino', 'Outro']

    for index, funcionario in funcionarios_df.iterrows():
        id_fun = funcionario['ID_fun']
        data_nasc_funcionario = funcionario['Data_nascimento']

        if random.random() < 0.7:
            num_dependentes = random.randint(1, 4)  # Aumentei para 4 para ter mais chance de criar duplicados para limpar

            for _ in range(num_dependentes):
                parentesco = random.choice(parentescos_possiveis)

                # Lógica para gerar data de nascimento conforme o parentesco
                if parentesco == 'Pai/Mãe':
                    # Calcular idade atual do funcionário
                    idade_funcionario = (date.today() - data_nasc_funcionario).days // 365
                    idade_minima_pai = idade_funcionario + 18
                    idade_maxima_pai = min(idade_funcionario + 50, 100)  # Limitar a 100 anos de idade
                    # Garantir limites válidos (se o funcionário for muito jovem, ajusta)
                    if idade_minima_pai > idade_maxima_pai:
                        idade_minima_pai = idade_maxima_pai - 1 if idade_maxima_pai > 0 else 18
                    data_nascimento = faker.date_of_birth(minimum_age=idade_minima_pai, maximum_age=idade_maxima_pai)
                elif parentesco == 'Filho(a)':
                    # Filho deve ser mais novo: gerar data depois da do funcionário
                    start_child = data_nasc_funcionario + timedelta(days=1)
                    # Se por algum motivo o funcionário nasceu "hoje", o filho terá a mesma regra +1 dia
                    if start_child >= date.today():
                        data_nascimento = start_child
                    else:
                        # Usa API explícita para limites de datas
                        data_nascimento = faker.date_between_dates(date_start=start_child, date_end=date.today())
                else:
                    # Outros dependentes podem ter qualquer idade plausível (0 a 80 anos de idade agora)
                    data_nascimento = faker.date_of_birth(minimum_age=0, maximum_age=80)

                dependente_record = {
                    "ID_Fun": id_fun,
                    "nome": faker.name(),
                    "sexo": random.choice(sexos_possiveis),
                    "data_nascimento": data_nascimento,
                    "parentesco": parentesco
                }
                lista_dependentes.append(dependente_record)

    # --- PASSO DE LIMPEZA ---

    # 1. Criar o DataFrame com os dados
    if not lista_dependentes:
        return pd.DataFrame()  # Retorna um DataFrame vazio se nenhum dependente foi gerado

    dependentes_bruto_df = pd.DataFrame(lista_dependentes)

    # 2. Separar os pais/mães dos outros dependentes
    pais_maes_df = dependentes_bruto_df[dependentes_bruto_df['parentesco'] == 'Pai/Mãe']
    outros_dependentes_df = dependentes_bruto_df[dependentes_bruto_df['parentesco'] != 'Pai/Mãe']

    # 3. Remover duplicados APENAS da parte dos pais/mães
    pais_maes_unicos_df = pais_maes_df.drop_duplicates(subset=['ID_Fun', 'parentesco'], keep='first')

    # 4. Juntar novamente as duas partes para ter o DataFrame final e limpo
    dependentes_final_df = pd.concat([pais_maes_unicos_df, outros_dependentes_df], ignore_index=True)

    return dependentes_final_df

In [16]:
Dependentes = criar_dependentes(Funcionarios)
Dependentes

,ID_Fun,nome,sexo,data_nascimento,parentesco
0,6,Alexandre Paiva-Macedo,Outro,1941-07-31,Pai/Mãe
1,7,Sandro Mota-Campos,Feminino,1949-06-11,Pai/Mãe
2,11,Martim Gaspar,Feminino,1957-10-17,Pai/Mãe
3,12,Ivo Henriques,Outro,1941-10-31,Pai/Mãe
4,15,Rui Machado,Masculino,1955-04-02,Pai/Mãe
...,...,...,...,...,...
172,97,Francisco Moreira,Outro,1983-06-03,Cônjuge
173,99,Carlota Barbosa,Outro,2013-08-08,Filho(a)
174,100,Maria Antunes,Feminino,1963-06-14,Cônjuge
175,100,Eva Rodrigues,Outro,2025-09-12,Cônjuge


In [17]:
def criar_faltas(funcionarios_df):
    """
    Gera um histórico de faltas para os funcionários.
    Cada funcionário terá entre 0 e 10 faltas registadas.
    """
    lista_faltas = []
    justificacoes_comuns = ["Consulta médica","Doença súbita","Apoio à família","Assuntos pessoais urgentes","Procedimentos legais/burocráticos","Avaria de transporte", None, None] # Para simular faltas injustificadas

    # Itera sobre cada funcionário para criar o seu histórico de faltas
    for index, funcionario in funcionarios_df.iterrows():
        id_fun = funcionario['ID_fun']
        
        # Cada funcionário terá um número aleatório de faltas (de 0 a 10)
        num_faltas = int(np.random.normal(loc=3, scale=2))
        
        # Para evitar que as faltas do mesmo funcionário sejam todas no mesmo dia,
        # geramos um conjunto de datas únicas para este funcionário
        datas_possiveis = [faker.date_between(start_date='-3y', end_date='today') for _ in range(num_faltas)]
        datas_unicas = set(datas_possiveis) # 'set' remove datas duplicadas automaticamente
        
        for data_falta in datas_unicas:
            falta_record = {
                "ID_Fun": id_fun,
                "Data": data_falta,
                "Justificacao": random.choice(justificacoes_comuns)
            }
            lista_faltas.append(falta_record)
            
    return pd.DataFrame(lista_faltas)

In [18]:
Faltas = criar_faltas(Funcionarios)
Faltas

,ID_Fun,Data,Justificacao
0,2,2023-06-12,Assuntos pessoais urgentes
1,2,2022-11-27,None
2,2,2023-11-27,Procedimentos legais/burocráticos
3,5,2024-03-05,Procedimentos legais/burocráticos
4,6,2025-10-04,Apoio à família
...,...,...,...
253,100,2024-02-08,None
254,100,2025-10-13,Consulta médica
255,100,2024-03-18,Consulta médica
256,100,2025-10-15,Procedimentos legais/burocráticos


In [19]:
def criar_historico_empresas(funcionarios_df):
    """
    Gera um histórico profissional para os funcionários.
    O primeiro registo (i=0) é SEMPRE o emprego atual na 'bd054' com Data_fim NULL.
    Os registos seguintes (i>0) são empregos anteriores noutras empresas.
    """
    lista_historico = []
    
    
    # Itera sobre cada funcionário
    for index, funcionario in funcionarios_df.iterrows():
        id_fun = funcionario['ID_fun']
        
        # 80% de chance de o funcionário ter um histórico (além do atual)
        # Nota: Agora todos terão PELO MENOS 1 registo (o atual)
        num_historicos = random.randint(0, 3) # 0 = só tem o emprego atual

        # Âncora de data. Começa com a data de início do emprego atual
        # A data de início na 'bd054' é algures nos últimos 5 anos
        data_inicio_atual = faker.date_between(start_date='-5y', end_date='today')
        
        # A âncora para empregos anteriores será ANTES desta data
        latest_end_date = data_inicio_atual - timedelta(days=random.randint(30, 365)) # Intervalo entre empregos
            
        for i in range(num_historicos + 1): # +1 para incluir o emprego atual
            
            # --- MUDANÇA CRÍTICA AQUI ---
            if i == 0:
                # O primeiro registo é SEMPRE o emprego atual na bd054
                nome_empresa = 'bd054'
                data_fim = None # Correto, como você disse!
                data_inicio = data_inicio_atual
                # Usamos o cargo real do funcionário na nossa empresa
                cargo = funcionario['Cargo']

            else:
                # Empregos anteriores (i > 0)
                nome_empresa = faker.company()
                data_fim = latest_end_date # Terminou antes do emprego anterior
                data_inicio = data_fim - timedelta(days=random.randint(365, 365 * 5)) # Durou 1-5 anos
                cargo = faker.job()
                
                # Prepara a âncora para o próximo loop (o emprego ainda mais antigo)
                latest_end_date = data_inicio - timedelta(days=random.randint(30, 365))
            
            historico_record = {
                "ID_Fun": id_fun,
                "Nome_empresa": nome_empresa,
                "Cargo": cargo,
                "Data_inicio": data_inicio,
                "Data_fim": data_fim
            }
            lista_historico.append(historico_record)

    return pd.DataFrame(lista_historico)

In [20]:
Historico_empresas = criar_historico_empresas(Funcionarios)
Historico_empresas

,ID_Fun,Nome_empresa,Cargo,Data_inicio,Data_fim
0,1,bd054,Executivo de Contas,2021-11-21,None
1,1,Nunes Cardoso Lda.,Empregado das agências de viagem,2019-12-04,2021-01-07
2,1,Assunção Assunção Lda.,Secretário da área jurídica,2014-07-07,2019-04-26
3,2,bd054,Representante de Vendas Interno,2024-10-27,None
4,3,bd054,Analista de Qualidade,2025-08-02,None
...,...,...,...,...,...
250,98,Sousa,Calceteiro,2018-04-02,2020-06-20
251,99,bd054,Assistente Administrativo,2021-12-03,None
252,99,Pires Morais e Filhos,"Encarregados das indústrias da pasta, papel, i...",2018-12-27,2021-03-03
253,100,bd054,Especialista em Sucesso do Cliente,2024-02-26,None


In [21]:
def criar_modulo_recrutamento(funcionarios_df, departamentos_df, num_candidatos=50, num_vagas=20):
    """
    Gera dados para as tabelas Candidatos, Vagas, Requisitos_vaga e Candidato_a.
    """
    lista_candidatos = []
    lista_vagas = []
    lista_requisitos = []
    lista_candidaturas = []

    # --- 1. Gerar Candidatos ---
    for i in range(1, num_candidatos + 1):
        candidato_record = {
            "ID_cand": i,
            "Nome": faker.name(),
            "Email": faker.unique.email(),
            "Telemovel": faker.phone_number(),
            "CV": None, # BLOBs são representados como None
            "carta_motivacao": None
        }
        lista_candidatos.append(candidato_record)
    candidatos_df = pd.DataFrame(lista_candidatos)
    ids_candidatos_disponiveis = candidatos_df['ID_cand'].tolist()

    # --- 2. Gerar Vagas ---
    ids_departamentos_disponiveis = departamentos_df['ID_depart'].tolist()
    estados_vaga = ['Aberta', 'Fechada', 'Suspensa']

    for i in range(1, num_vagas + 1):
        vaga_record = {
            "ID_vaga": i,
            "Data_abertura": faker.date_between(start_date='-2y', end_date='today'),
            "Estado": random.choice(estados_vaga),
            "ID_depart": random.choice(ids_departamentos_disponiveis)
        }
        lista_vagas.append(vaga_record)
    vagas_df = pd.DataFrame(lista_vagas)
    
    # --- 3. Gerar Requisitos para cada Vaga ---
    requisitos_possiveis = [
        'Python', 'SQL', 'Gestão de Projetos', 'Comunicação', 'Inglês Fluente',
        'Análise de Dados', 'React', 'Liderança', 'Trabalho em Equipa', 'Excel Avançado',
        'Marketing Digital', 'Contabilidade', 'Java', 'Atendimento ao Cliente'
    ]
    for index, vaga in vagas_df.iterrows():
        # Cada vaga terá entre 2 e 5 requisitos
        num_requisitos = random.randint(2, 5)
        # Usamos random.sample para garantir que os requisitos para a mesma vaga são únicos
        requisitos_para_vaga = random.sample(requisitos_possiveis, num_requisitos)
        
        for req in requisitos_para_vaga:
            requisito_record = {
                "ID_vaga": vaga['ID_vaga'],
                "Requisito": req
            }
            lista_requisitos.append(requisito_record)
    requisitos_df = pd.DataFrame(lista_requisitos)

    # --- 4. Gerar Candidaturas (Candidato_a) ---
    # Para ser mais realista, vamos pegar nos IDs dos funcionários de RH para serem os recrutadores
    id_depart_rh = departamentos_df[departamentos_df['Nome'] == 'Recursos Humanos']['ID_depart'].iloc[0]
    ids_recrutadores = funcionarios_df[funcionarios_df['ID_depart'] == id_depart_rh]['ID_fun'].tolist()
    # Fallback: se não houver ninguém em RH, usamos qualquer funcionário
    if not ids_recrutadores:
        ids_recrutadores = funcionarios_df['ID_fun'].tolist()
        
    estados_candidatura = ['Submetido', 'Em análise', 'Entrevista', 'Rejeitado', 'Rejeitado', 'Rejeitado', 'Contratado']

    # Cada candidato vai candidatar-se a um número aleatório de vagas (0 a 3)
    for id_candidato in ids_candidatos_disponiveis:
        num_candidaturas = random.randint(0, 3)
        if num_candidaturas > 0:
            # Escolhe 'num_candidaturas' vagas aleatórias e únicas para se candidatar
            vagas_aplicadas = vagas_df.sample(num_candidaturas)
            
            for index, vaga in vagas_aplicadas.iterrows():
                # A data da candidatura tem de ser posterior à data de abertura da vaga
                data_candidatura = faker.date_between(start_date=vaga['Data_abertura'], end_date='today')
                
                candidatura_record = {
                    "ID_cand": id_candidato,
                    "ID_Vaga": vaga['ID_vaga'],
                    "Data_cand": data_candidatura,
                    "Estado": random.choice(estados_candidatura),
                    "ID_recrutador": random.choice(ids_recrutadores)
                }
                lista_candidaturas.append(candidatura_record)
    candidato_a_df = pd.DataFrame(lista_candidaturas)
    
    return candidatos_df, vagas_df, requisitos_df, candidato_a_df

In [22]:
Candidatos, Vagas, Requisitos_vaga, Candidato_a = criar_modulo_recrutamento(Funcionarios, Departamentos)

In [23]:
Candidatos

,ID_cand,Nome,Email,Telemovel,CV,carta_motivacao
0,1,Luís Anjos-Esteves,jbatista@example.net,(351) 936 730 511,None,None
1,2,Mafalda do Andrade,vazmelissa@example.net,936523565,None,None
2,3,Erica Simões,paulo34@example.com,(351) 923 419 018,None,None
3,4,Fernando Cardoso,carolina08@example.org,(351) 967 794 786,None,None
4,5,Nádia-Gabriela Barros,ismael50@example.com,(351) 937 309 622,None,None
5,6,William Leite,brunomiranda@example.org,(351) 936 474 231,None,None
6,7,Leandro Alves,rafaelcosta@example.com,(351) 937 921 135,None,None
7,8,Anita Costa,anita35@example.net,(351) 913 330 615,None,None
8,9,Wilson do Oliveira,batistasalome@example.org,910678574,None,None
9,10,Mafalda Freitas,ftavares@example.com,(351) 215835414,None,None


In [24]:
Vagas

,ID_vaga,Data_abertura,Estado,ID_depart
0,1,2024-07-03,Aberta,6
1,2,2024-04-03,Fechada,7
2,3,2024-08-18,Aberta,6
3,4,2025-02-15,Suspensa,2
4,5,2024-10-11,Suspensa,8
5,6,2024-03-15,Aberta,5
6,7,2024-09-27,Aberta,4
7,8,2025-08-07,Aberta,6
8,9,2024-04-05,Suspensa,7
9,10,2025-07-06,Suspensa,3


In [25]:
Requisitos_vaga


,ID_vaga,Requisito
0,1,Inglês Fluente
1,1,Comunicação
2,1,Marketing Digital
3,1,Análise de Dados
4,2,Contabilidade
...,...,...
70,19,Atendimento ao Cliente
71,20,Inglês Fluente
72,20,Gestão de Projetos
73,20,Liderança


In [26]:
Candidato_a

,ID_cand,ID_Vaga,Data_cand,Estado,ID_recrutador
0,2,8,2025-10-04,Entrevista,58
1,2,20,2025-04-29,Em análise,98
2,3,10,2025-09-22,Contratado,88
3,3,13,2025-07-20,Rejeitado,62
4,5,2,2024-10-26,Contratado,57
...,...,...,...,...,...
69,45,20,2025-05-25,Submetido,37
70,45,5,2024-12-28,Rejeitado,8
71,47,6,2025-04-09,Rejeitado,52
72,47,20,2025-10-04,Rejeitado,57


In [27]:
def criar_catalogo_formacoes():
    """
    Cria um catálogo fixo e manual de 5 formações com descrições realistas.
    """
    dados_formacoes = [
        {"ID_for": 1, "Nome_formacao": 'Liderança e Gestão de Equipas', "Descricao": 'Desenvolver competências de liderança, motivação e gestão de conflitos para chefes de equipa.', "Data_inicio": date(2023, 10, 5), "Data_fim": date(2023, 10, 10), "Estado": 'Concluida'},
        {"ID_for": 2, "Nome_formacao": 'Cibersegurança para Colaboradores', "Descricao": 'Sensibilizar para as melhores práticas de segurança digital, phishing e proteção de dados.', "Data_inicio": date(2024, 3, 15), "Data_fim": date(2024, 3, 16), "Estado": 'Concluida'},
        {"ID_for": 3, "Nome_formacao": 'Análise de Dados com Python', "Descricao": 'Introdução à análise de dados utilizando as bibliotecas Pandas e Matplotlib.', "Data_inicio": date(2024, 9, 2), "Data_fim": date(2024, 9, 13), "Estado": 'Em curso'},
        {"ID_for": 4, "Nome_formacao": 'Técnicas de Negociação e Vendas', "Descricao": 'Aperfeiçoar técnicas de comunicação, persuasão e fecho de negócios para a equipa comercial.', "Data_inicio": date(2025, 2, 10), "Data_fim": date(2025, 2, 14), "Estado": 'Planeada'},
        {"ID_for": 5, "Nome_formacao": 'Inteligência Emocional no Trabalho', "Descricao": 'Gerir emoções, melhorar o autoconhecimento e a empatia nas relações interpessoais no trabalho.', "Data_inicio": date(2025, 5, 19), "Data_fim": date(2025, 5, 20), "Estado": 'Planeada'}
    ]
    
    return pd.DataFrame(dados_formacoes)

In [28]:
Formacoes = criar_catalogo_formacoes()
Formacoes

,ID_for,Nome_formacao,Descricao,Data_inicio,Data_fim,Estado
0,1,Liderança e Gestão de Equipas,"Desenvolver competências de liderança, motivaç...",2023-10-05,2023-10-10,Concluida
1,2,Cibersegurança para Colaboradores,Sensibilizar para as melhores práticas de segu...,2024-03-15,2024-03-16,Concluida
2,3,Análise de Dados com Python,Introdução à análise de dados utilizando as bi...,2024-09-02,2024-09-13,Em curso
3,4,Técnicas de Negociação e Vendas,"Aperfeiçoar técnicas de comunicação, persuasão...",2025-02-10,2025-02-14,Planeada
4,5,Inteligência Emocional no Trabalho,"Gerir emoções, melhorar o autoconhecimento e a...",2025-05-19,2025-05-20,Planeada


In [29]:
def criar_teve_formacao(funcionarios_df, formacoes_df):
    """
    Gera as inscrições dos funcionários nas formações.
    Garante que as datas de participação do funcionário estão DENTRO
    do período em que a formação esteve ativa.
    """
    lista_teve_formacao = []

    for index, funcionario in funcionarios_df.iterrows():
        if random.random() < 0.6:
            num_formacoes_feitas = random.randint(1, 2)
            formacoes_a_inscrever = formacoes_df.sample(num_formacoes_feitas)
            
            for _, formacao in formacoes_a_inscrever.iterrows():
                
                # --- LÓGICA DE DATA CORRIGIDA AQUI ---
                
                # 1. Definir os limites da formação
                formacao_inicio = formacao['Data_inicio']
                formacao_fim = formacao['Data_fim']
                
                # 2. Gerar uma data de início de participação DENTRO desse período
                # (usamos date_between_dates para trabalhar com objetos date)
                data_inicio_participacao = faker.date_between_dates(date_start=formacao_inicio, date_end=formacao_fim)
                
                # 3. Gerar uma data de fim de participação DENTRO do que resta do período
                data_fim_participacao = faker.date_between_dates(date_start=data_inicio_participacao, date_end=formacao_fim)
                
                # --- FIM DA CORREÇÃO ---

                teve_formacao_record = {
                    "ID_Fun": funcionario['ID_fun'],
                    "ID_for": formacao['ID_for'],
                    "Certificado": None, # BLOB
                    "Data_inicio": data_inicio_participacao, # Data corrigida
                    "Data_fim": data_fim_participacao      # Data corrigida
                }
                lista_teve_formacao.append(teve_formacao_record)

    return pd.DataFrame(lista_teve_formacao)

In [30]:
Teve_Formacao = criar_teve_formacao(Funcionarios, Formacoes)
Teve_Formacao

,ID_Fun,ID_for,Certificado,Data_inicio,Data_fim
0,2,3,None,2024-09-11,2024-09-11
1,2,4,None,2025-02-10,2025-02-11
2,4,1,None,2023-10-05,2023-10-09
3,4,3,None,2024-09-03,2024-09-03
4,5,1,None,2023-10-08,2023-10-09
...,...,...,...,...,...
75,97,3,None,2024-09-12,2024-09-12
76,98,2,None,2024-03-15,2024-03-15
77,99,2,None,2024-03-15,2024-03-15
78,100,3,None,2024-09-11,2024-09-12


In [31]:
FRASES_CRITERIOS = [
    "Avaliação da pontualidade e assiduidade no último semestre.",
    "Desempenho nos projetos de equipa e capacidade de colaboração.",
    "Capacidade de resolução de problemas e tomada de decisão.",
    "Qualidade da comunicação com o cliente e stakeholders internos.",
    "Iniciativa e proatividade na sugestão de novas melhorias.",
    "Qualidade técnica do trabalho entregue e atenção ao detalhe.",
    "Gestão de tempo, cumprimento de prazos e organização."
]

FRASES_AUTOAVALIACAO = [
    "Sinto que atingi todos os meus objetivos principais este semestre.",
    "Completei a formação X e sinto-me mais confiante nas minhas tarefas.",
    "Gostaria de ter mais formação na área de análise de dados para contribuir melhor.",
    "Consegui melhorar a minha organização e gestão de tempo, como foi discutido.",
    "Tive alguma dificuldade em gerir o projeto Y, mas aprendi muito com o processo.",
    "Preciso de focar-me em ser mais conciso nas reuniões de equipa."
]


def criar_avaliacoes(funcionarios_df, num_avaliacoes=40):
    """
    Gera um número fixo de avaliações (ex: 40) com frases realistas,
    IDs de avaliadores inteiros e uma avaliação numérica.

    Notas:
    - Avaliacao_Numerica é um inteiro de 1 a 5 (escala Likert). Pode ser ajustado conforme necessário.
    """
    lista_avaliacoes = []
    
    # Obter uma lista de todos os IDs de funcionários
    ids_funcionarios_disponiveis = funcionarios_df['ID_fun'].tolist()

    # Se não houver funcionários, não podemos fazer nada
    if not ids_funcionarios_disponiveis or len(ids_funcionarios_disponiveis) < 2:
        print("Aviso: É preciso ter pelo menos 2 funcionários para gerar avaliações.")
        return pd.DataFrame()

    for _ in range(num_avaliacoes):
        
        # 1. Escolhe um funcionário para ser AVALIADO
        id_fun_avaliado = random.choice(ids_funcionarios_disponiveis)
        
        # 2. Escolhe um AVALIADOR
        id_avaliador = random.choice(ids_funcionarios_disponiveis)
        
        # 3. Garante que não é a mesma pessoa (autoavaliação é noutro campo)
        while id_avaliador == id_fun_avaliado:
            id_avaliador = random.choice(ids_funcionarios_disponiveis)
            
        # 4. Gera uma data única para a chave primária
        data_avaliacao = faker.date_between(start_date='-3y', end_date='today')

        # 5. 70% de chance de ter uma autoavaliação
        autoavaliacao = random.choice(FRASES_AUTOAVALIACAO) if random.random() < 0.7 else None

        # 6. Avaliação numérica (1 a 5)
        avaliacao_numerica = random.randint(1, 5)
        
        avaliacao_record = {
            "ID_Fun": id_fun_avaliado,
            "ID_avaliador": id_avaliador, # Isto será sempre um INT
            "Data": data_avaliacao,
            "Avaliacao": None, # BLOB
            "Criterios": random.choice(FRASES_CRITERIOS), # Frase realista
            "Autoavaliacao": autoavaliacao, # Frase realista ou None
            "Avaliacao_Numerica": avaliacao_numerica
        }
        
        lista_avaliacoes.append(avaliacao_record)

    # Criar o DataFrame
    avaliacoes_df = pd.DataFrame(lista_avaliacoes)

    # Salvaguarda: Remove duplicados na chave primária (caso tenhamos gerado o mesmo trio por acaso)
    avaliacoes_df = avaliacoes_df.drop_duplicates(subset=['ID_Fun', 'ID_avaliador', 'Data'], keep='first')
    return avaliacoes_df

In [32]:
Avaliacoes = criar_avaliacoes(Funcionarios)
Avaliacoes

,ID_Fun,ID_avaliador,Data,Avaliacao,Criterios,Autoavaliacao,Avaliacao_Numerica
0,70,32,2023-02-15,None,Desempenho nos projetos de equipa e capacidade...,None,2
1,35,5,2025-09-25,None,Avaliação da pontualidade e assiduidade no últ...,None,2
2,96,20,2024-07-20,None,Desempenho nos projetos de equipa e capacidade...,Gostaria de ter mais formação na área de análi...,1
3,73,59,2025-03-18,None,Capacidade de resolução de problemas e tomada ...,Completei a formação X e sinto-me mais confian...,4
4,61,85,2023-04-23,None,Qualidade técnica do trabalho entregue e atenç...,Gostaria de ter mais formação na área de análi...,4
5,34,19,2024-12-31,None,Iniciativa e proatividade na sugestão de novas...,Gostaria de ter mais formação na área de análi...,3
6,92,81,2022-12-09,None,Avaliação da pontualidade e assiduidade no últ...,Gostaria de ter mais formação na área de análi...,3
7,11,20,2024-12-09,None,Avaliação da pontualidade e assiduidade no últ...,None,1
8,68,45,2024-01-17,None,Capacidade de resolução de problemas e tomada ...,"Tive alguma dificuldade em gerir o projeto Y, ...",3
9,18,92,2022-12-03,None,Iniciativa e proatividade na sugestão de novas...,"Tive alguma dificuldade em gerir o projeto Y, ...",1


In [33]:
def criar_utilizadores_permissoes(funcionarios_df):
    """
    Gera dados para as tabelas Utilizadores e Permissoes.
    Cria um utilizador para cada funcionário e atribui permissões aleatórias.
    """
    lista_utilizadores = []
    lista_permissoes = []
    
    # A sua lista de permissões válidas
    permissoes_possiveis = ['Funcionario', 'Manager', 'RH', 'Admin', 'Financeiro']

    # Itera sobre CADA funcionário para criar uma conta de utilizador
    for index, funcionario in funcionarios_df.iterrows():
        id_fun = funcionario['ID_fun']
        
        # --- 1. Criar o registo na tabela Utilizadores ---
        utilizador_record = {
            "ID_Fun": id_fun,
            # Gera uma password aleatória. 'special_chars=False' para caber no VARCHAR(30)
            "Password": faker.password(length=12, special_chars=False) 
        }
        lista_utilizadores.append(utilizador_record)
        
        # --- 2. Criar os registos na tabela Permissoes ---
        
        # Cada utilizador terá entre 1 e 3 permissões
        num_permissoes = random.randint(1, 3)
        
        # Escolhe 'num_permissoes' permissões únicas da lista
        permissoes_para_este_user = random.sample(permissoes_possiveis, num_permissoes)
        
        # Cria um registo para cada permissão atribuída
        for perm in permissoes_para_este_user:
            permissao_record = {
                "ID_Fun": id_fun,
                "Permissao": perm
            }
            lista_permissoes.append(permissao_record)

    utilizadores_df = pd.DataFrame(lista_utilizadores)
    permissoes_df = pd.DataFrame(lista_permissoes)
    
    return utilizadores_df, permissoes_df

In [34]:
Utilizadores, Permissoes = criar_utilizadores_permissoes(Funcionarios)

In [35]:
Utilizadores

,ID_Fun,Password
0,1,b806ySSPXKHL
1,2,d3mNnzafPni4
2,3,d89Wyask5M6y
3,4,SCGdjb9CQr98
4,5,sGpNqTann5ah
...,...,...
95,96,XXAmES6m3D7B
96,97,7YQiwj3XF4L7
97,98,ZQLgICdhZ8ha
98,99,2K6DTdOEk2DP


In [36]:
Permissoes

,ID_Fun,Permissao
0,1,Admin
1,2,Manager
2,2,RH
3,3,Financeiro
4,3,Manager
...,...,...
193,99,Financeiro
194,99,Admin
195,100,RH
196,100,Financeiro


In [37]:
import pandas as pd
from datetime import date, datetime
import math

print("A iniciar a geração do ficheiro 'dados_insercao.sql' (um INSERT por tabela)...")

# --- Função Auxiliar (sem alterações) ---
def formatar_valor_sql(valor):
    if pd.isna(valor) or valor is None:
        return "NULL"
    if isinstance(valor, str):
        return "'" + valor.replace("'", "''") + "'"
    if isinstance(valor, (int, float)):
        return str(valor)
    if isinstance(valor, (date, datetime)):
        return f"'{valor.isoformat()}'"
    return f"'{str(valor)}'"

# --- Dicionário com todos os DataFrames (sem alterações) ---
todos_dataframes = {
    "Remuneracoes": Remuneracoes,
    "Salario": Salarios,
    "Beneficios": Beneficios,
    "Ferias": Ferias,
    "Dependentes": Dependentes,
    "Faltas": Faltas,
    "Historico_empresas": Historico_empresas,
    "Candidatos": Candidatos,
    "Vagas": Vagas,
    "Candidato_a": Candidato_a,
    "Requisitos_vaga": Requisitos_vaga,
    "Formacoes": Formacoes,
    "Teve_Formacao": Teve_Formacao,
    "Avaliacoes": Avaliacoes,
    "Utilizadores": Utilizadores,
    "Permissoes": Permissoes
}

# --- NOVO: Função para gerar UM ÚNICO INSERT por tabela ---
def escrever_insert_unico_sql(f, df, nome_tabela, colunas_sql=None):
    """
    Escreve um DataFrame num ficheiro SQL usando UMA ÚNICA instrução INSERT.
    """
    if df.empty:
        f.write(f"\n-- Tabela {nome_tabela} está vazia. A saltar.\n")
        return

    f.write(f"\n-- Dados para {nome_tabela}\n")
    
    if colunas_sql:
        cols_sql = colunas_sql
    else:
        cols_sql = ", ".join(df.columns)
    
    # Escreve o início da instrução
    f.write(f"INSERT INTO {nome_tabela} ({cols_sql}) VALUES\n")
    
    linhas_de_valores = []
    # Itera por TODAS as linhas do DataFrame
    for row in df.itertuples(index=False, name=None):
        valores_formatados = f"({', '.join([formatar_valor_sql(val) for val in row])})"
        linhas_de_valores.append(valores_formatados)
    
    # Junta todas as linhas de valores com uma vírgula
    f.write(",\n".join(linhas_de_valores))
    f.write(";\n") # Termina a instrução INSERT

# --- Processo de Escrita do Ficheiro (Atualizado) ---
try:
    with open('dados_insercao.sql', 'w', encoding='utf-8') as f:
        f.write("-- Script de Inserção de Dados Gerado Automaticamente\n")

        # 1. Departamentos (Inserir com gerente NULL)
        df_deptos_temp = Departamentos[['ID_depart', 'Nome']].copy()
        df_deptos_temp['ID_gerente'] = None
        escrever_insert_unico_sql(f, df_deptos_temp, "Departamentos", colunas_sql="ID_depart, Nome, ID_gerente")
        
        # 2. Funcionarios
        escrever_insert_unico_sql(f, Funcionarios, "Funcionarios")

        # 3. UPDATE Departamentos (Agora com os gerentes)
        f.write("\n-- 3. Atualização dos Gerentes nos Departamentos\n")
        for row in Departamentos.itertuples(index=False):
            if pd.notna(row.ID_gerente):
                f.write(f"UPDATE Departamentos SET ID_gerente = {int(row.ID_gerente)} WHERE ID_depart = {row.ID_depart};\n")

        # 4. Inserção de todas as outras tabelas
        f.write("\n-- 4. Inserção das restantes tabelas\n")
        for nome_tabela, df in todos_dataframes.items():
            
            # Lógica especial para Candidato_a para renomear colunas
            if nome_tabela == 'Candidato_a':
                escrever_insert_unico_sql(f, df, nome_tabela, colunas_sql="ID_cand, ID_Vaga, Data_cand, Estado, ID_recrutador")
            else:
                escrever_insert_unico_sql(f, df, nome_tabela)


    print("O ficheiro 'dados_insercao.sql' foi gerado com sucesso.")

except Exception as e:
    print(f"\n--- ERRO ---")
    print(f"Ocorreu um erro ao gerar o ficheiro: {e}")
    print("Verifique os nomes dos seus DataFrames.")

A iniciar a geração do ficheiro 'dados_insercao.sql' (um INSERT por tabela)...
O ficheiro 'dados_insercao.sql' foi gerado com sucesso.
